## Step 1. Prepare Dataset

### 1.1 Loading the Image Data

In [1]:
import os

# Change to your desired directory
os.chdir('/common/users/sk2779/df_words') # change $USER to netid

# Confirm it's changed
print("Current directory:", os.getcwd())

Current directory: /common/users/sk2779/df_words


In [2]:
import pickle

with open('./dfwords_0_20000.pkl', 'rb') as file:
    loaded_dfwords1 = pickle.load(file)

with open('./dfwords_20000_40000.pkl', 'rb') as file:
    loaded_dfwords2 = pickle.load(file)

with open('./dfwords_40000_60000.pkl', 'rb') as file:
    loaded_dfwords3 = pickle.load(file)

with open('./dfwords_60000_80000.pkl', 'rb') as file:
    loaded_dfwords4 = pickle.load(file)

with open('./dfwords_80000_100000.pkl', 'rb') as file:
    loaded_dfwords5 = pickle.load(file)

with open('./dfwords_100000_120000.pkl', 'rb') as file:
    loaded_dfwords6 = pickle.load(file)

with open('./dfwords_120000_140000.pkl', 'rb') as file:
    loaded_dfwords7 = pickle.load(file)

with open('./dfwords_140000_160000.pkl', 'rb') as file:
    loaded_dfwords8 = pickle.load(file)

with open('./dfwords_160000_180000.pkl', 'rb') as file:
    loaded_dfwords9 = pickle.load(file)

with open('./dfwords_180000_200000.pkl', 'rb') as file:
    loaded_dfwords10 = pickle.load(file)

with open('./dfwords_200000_227055.pkl', 'rb') as file:
    loaded_dfwords11 = pickle.load(file)

In [3]:
import pandas as pd

loaded_dfwords = pd.concat([loaded_dfwords1, loaded_dfwords2], ignore_index=True)  # vertical stack
loaded_dfwords = pd.concat([loaded_dfwords, loaded_dfwords3], ignore_index=True)  # vertical stack
loaded_dfwords = pd.concat([loaded_dfwords, loaded_dfwords4], ignore_index=True)  # vertical stack
loaded_dfwords = pd.concat([loaded_dfwords, loaded_dfwords5], ignore_index=True)  # vertical stack
loaded_dfwords = pd.concat([loaded_dfwords, loaded_dfwords6], ignore_index=True)  # vertical stack
loaded_dfwords = pd.concat([loaded_dfwords, loaded_dfwords7], ignore_index=True)  # vertical stack
loaded_dfwords = pd.concat([loaded_dfwords, loaded_dfwords8], ignore_index=True)  # vertical stack
loaded_dfwords = pd.concat([loaded_dfwords, loaded_dfwords9], ignore_index=True)  # vertical stack
loaded_dfwords = pd.concat([loaded_dfwords, loaded_dfwords10], ignore_index=True)  # vertical stack
loaded_dfwords = pd.concat([loaded_dfwords, loaded_dfwords11], ignore_index=True)  # vertical stack

In [4]:
loaded_dfwords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227055 entries, 0 to 227054
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      227055 non-null  object
 1   text    227055 non-null  object
 2   image   227055 non-null  object
dtypes: object(3)
memory usage: 5.2+ MB


In [5]:
loaded_dfwords.head(3)

,id,text,image
0,7u2pNft,4,"[[[211, 209, 208], [210, 208, 207], [211, 209,..."
1,EHXbhtL,↳,"[[[187, 166, 139], [188, 167, 140], [190, 169,..."
2,EHXbhtL,co,"[[[188, 164, 134], [188, 164, 134], [189, 165,..."


### 1.2 Loading the Testing Data

In [6]:
import os

# Change to your desired directory
os.chdir('/common/home/sk2779/courses/cs439/stylized-ocr') # change $USER to netid

# Confirm it's changed
print("Current directory:", os.getcwd())

Current directory: /common/home/sk2779/courses/cs439/stylized-ocr


In [7]:
df_test_info= pd.read_csv("dataset_info/df_test_info.csv") 

In [8]:
df_test_info.head()

,word_idx,id,text
0,0,7u2pNft,4
1,274,FXwDU08,Funk
2,275,FXwDU08,adelic
3,276,FXwDU08,or
4,277,FXwDU08,Brain


In [9]:
df_test_info_premerge= df_test_info.drop(['id', 'text'], axis=1)

In [10]:
df_test= pd.merge(
    loaded_dfwords.reset_index(), 
    df_test_info_premerge, 
    left_on='index', 
    right_on='word_idx',
    how='right'  # Keeps all rows from df1
)

In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40400 entries, 0 to 40399
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     40400 non-null  int64 
 1   id        40400 non-null  object
 2   text      40400 non-null  object
 3   image     40400 non-null  object
 4   word_idx  40400 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 1.5+ MB


In [12]:
df_test.head()

,index,id,text,image,word_idx
0,0,7u2pNft,4,"[[[211, 209, 208], [210, 208, 207], [211, 209,...",0
1,274,FXwDU08,Funk,"[[[126, 145, 152], [124, 143, 150], [121, 140,...",274
2,275,FXwDU08,adelic,"[[[136, 154, 165], [140, 159, 169], [133, 152,...",275
3,276,FXwDU08,or,"[[[157, 177, 188], [157, 177, 188], [157, 177,...",276
4,277,FXwDU08,Brain,"[[[180, 198, 215], [179, 197, 215], [181, 198,...",277


## Step 2. Make Inference of the IAM-fine-tuned Base Model

### 2.1 Setup Target Model

In [13]:
# get targetted model
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transfor

### 2.2 Do Inference- OCRing

In [15]:
from tqdm import tqdm
from PIL import Image
def readText_batch(df, indices, model, processor):
    """Process multiple images at once"""
    images= [df['image'][idx]for idx in indices]
    pixel_values = processor(images=images, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)
    return processor.batch_decode(generated_ids, skip_special_tokens=True)

def process_all_rows_batched(df, model, processor, batch_size=8):
    results = []
    for i in tqdm(range(0, len(df), batch_size), desc="Processing batches"):
        batch_indices = range(i, min(i+batch_size, len(df)))
        try:
            batch_texts = readText_batch(df, batch_indices,model, processor)
            for idx, text in zip(batch_indices, batch_texts):
                results.append({
                    'id': df['id'][idx],
                    'true_text': df['text'][idx],
                    'predicted_text': text
                })
        except Exception as e:
            print(f"Error in batch {i//batch_size}: {str(e)}")
            for idx in batch_indices:
                results.append({
                    'id': df['id'][idx],
                    'true_text': df['text'][idx],
                    'predicted_text': None,
                    'error': str(e)
                })
    return pd.DataFrame(results)

# Usage (best for GPU utilization)
results_df = process_all_rows_batched(df_test, model_iam, processor_iam)

Processing batches: 100%|██████████| 13/13 [00:37<00:00,  2.89s/it]


### Step 3. Evalute its performance

### 3.1 Define Metrics

In [21]:
from tqdm import tqdm
from evaluate import load
cer = load("cer")

def compute_metrics(pred_str, label_str):
    pred_str=pred_str.strip()
    label_str=label_str.strip()
    try: 
        score = cer.compute(predictions=[pred_str], references=[label_str])
        return score
    except Exception as e:
        print("error", e)
        print(type(pred_str), len(pred_str), pred_str)
        print(type(label_str), len(label_str), label_str)
        return None

In [22]:
from tqdm import tqdm
tqdm.pandas()  # Enable progress_apply for pandas

results_df["metrics"] = results_df.progress_apply(
    lambda row: compute_metrics(row["predicted_text"], row["true_text"]),
    axis=1
)

100%|██████████| 100/100 [00:00<00:00, 156.41it/s]


In [23]:
results_df.head()

,id,true_text,predicted_text,metrics
0,7u2pNft,4,4,0.000000
1,FXwDU08,Funk,""" Ferripezi",2.500000
2,FXwDU08,adelic,indel effect,1.333333
3,FXwDU08,or,odir .,2.000000
4,FXwDU08,Brain,Brain,0.000000


In [ ]:
results_df.to_csv("eval/model_words_base.csv")